# Plot IM Lup: ALMA vs SPHERE

In [ ]:
from pathlib import Path
import requests
import os

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from scipy.spatial.transform import Rotation as R
from scipy.integrate import cumtrapz
from seaborn.cm import rocket, mako
from astropy.io import fits

sqdeg_per_sr = 4 * np.pi**2 / 360.0**2

import helper
plt.style.use([helper.araa_style])

Paths to the data files

In [ ]:
profile_url = 'https://almascience.eso.org/almadata/lp/DSHARP/profiles/IMLup.profile.txt'  # Path to the DSHARP radial profile
fname_S = 'data_IMLup/IM_Lup_reducedRob_median_Hband_12.25mas_mod.fits'
fname_A = 'data_IMLup/IMLup_continuum.fits'
fname_CO = '/Users/birnstiel/DATA/IM_Lup_CO_220GHz/IM_Lup_CO_220GHz.0.3arcsec.JvMcorr.image.fits'

## Get $r_{90}$

Download and load the data

In [ ]:
req = requests.get(profile_url, stream=True)

fname = os.path.join('data_IMLup', Path(profile_url).name)

with open(fname, 'w') as fh:
    fh.write(req.text)

profile = np.loadtxt(fname)

prof_r = profile[:, 1]
prof_I = profile[:, 2]

compute the 90% radius

In [ ]:
cumflux = cumtrapz(2 * np.pi * prof_I * prof_r, x=prof_r, initial=0)
r_90 = np.interp(90, cumflux / cumflux[-1], prof_r)

## Image & Plotting Parameters

In [ ]:
clip = 3.0       # at how many arcsec to crop the image
drange = 1e-3    # dynamic range of the image
bar_in_au = 100. # how many au the size bar in the image has

# get the distance in parsec based on the GAIA parallax

gaia_plx = 6.4174937230476985 * u.mas
dpc = (u.au / np.tan(gaia_plx)).to('pc').value

Rings measured by Avenhaus et al. 2018

In [ ]:
r_HA = [0.58, 0.96, 1.52, 2.10]
inc_HA = [53., 55, 55., 56.]
PA_HA = 325.
hr_HA =  [0.18, 0.18, 0.23, 0.25]

First two rings are measured by Huang et al. 2018. Last one is my by-eye fit of the outer edge using the average inclination of the other rings

In [ ]:
r_JH = [0.743, 0.845, r_90]
inc_JH = 47.5
PA_JH = [145.2, 143.9]
PA_JH += [np.mean(PA_JH)]

## Load and plot images

In [ ]:
img_S = helper.image(fname_S, clip=clip)
img_A = helper.image(fname_A, clip=clip)
img_CO = helper.image(fname_CO, clip=clip)

### 2-Panel Plot: ALMA & SPHERE

In [ ]:
fig = plt.figure(figsize=(5, 2.3), dpi=300)
gs = fig.add_gridspec(ncols=3, nrows=1, width_ratios=[1, 1, 0.1], hspace=0.05)
axs = [fig.add_subplot(_gs) for _gs in gs]

for ax in axs[:-1]:
    ax.set_aspect('equal')
    ax.set_xlim(clip, -clip)
    ax.set_ylim(-clip, clip)
    ax.set_xlabel('$\Delta$ RA [arcsec]', fontdict=helper.label_font_dict)
    
# plot the two images

for ax, img, txt in zip(axs, [img_S, img_A], ['scattered light (1.65 $\mu$m)', 'thermal emission (1.25 mm)']):
    # image
    cc = ax.pcolormesh(img.xi, img.yi, np.maximum(img.data.T, drange), norm=LogNorm(drange, 1.0), rasterized=True)
    
    # scale bar
    ax.plot(-0.8 * clip * np.ones(2) + [0, bar_in_au / dpc], -0.82 * clip * np.ones(2), 'w', lw=3)
    ax.text(-0.8 * clip + bar_in_au / dpc / 2, -0.8 * clip, f'{bar_in_au:.0f} au',
            c='w', horizontalalignment='center', verticalalignment='bottom');
    ax.text(0.9 * clip, 0.9 * clip, txt, horizontalalignment='left', verticalalignment='top', c='w', fontsize='small')
    
# plot the rings in SPHERE

for i_ring in range(4):
    helper.plot_ring(axs[0], 
        r_HA[i_ring],
        hr_HA[i_ring] * r_HA[i_ring],
        np.deg2rad(inc_HA[i_ring]),
        np.deg2rad(90+PA_HA),
        nphi=50, c='w', alpha=0.25, lw=1, ls='--',  dashes=(5, 2))

# plot rings in ALMA
    
for i_ring in range(len(r_JH)):
    helper.plot_ring(axs[1], 
        r_JH[i_ring],
        0.0,
        np.deg2rad(inc_JH),
        np.deg2rad(90+PA_JH[i_ring]),
        nphi=50, c='w', alpha=0.25, lw=1, ls='--', dashes=(5, 2))


# adjust spacing
axs[1].yaxis.set_ticklabels([])
fig.subplots_adjust(wspace=0.02)
    
# colorbar
cb=plt.colorbar(cc, cax=axs[-1])
pos = axs[1].get_position()
axs[-1].set_position([pos.x1+0.02, pos.y0, pos.width / 15., pos.height])
cb.set_label(r'$I_\nu / I_\nu^{max}$', fontdict=helper.label_font_dict)

axs[0].set_ylabel('$\Delta$ Dec [arcsec]', fontdict=helper.label_font_dict);

In [ ]:
fig.savefig(helper.output_dir / 'imlup.pdf', transparent=True, bbox_inches='tight')

### Just CO

In [ ]:
drange=1e-1

f, ax = plt.subplots(dpi=150)
ax.set_aspect('equal')
cc = ax.pcolormesh(img_CO.xi, img_CO.yi, np.maximum(img_CO.data.T.max(0) / img_CO.data.T.max(), drange), rasterized=True,cmap=mako)

ax.set_xlim(clip, -clip)
ax.set_ylim(-clip, clip)
ax.set_xlabel('$\Delta$ RA [arcsec]', fontdict=helper.label_font_dict)

# colorbar
pos = ax.get_position()
cax = f.add_axes([pos.x1+0.02, pos.y0, pos.width / 15., pos.height])
cb=plt.colorbar(cc, cax=cax)

cb.set_label(r'$I_\nu / I_\nu^{max}$', fontdict=helper.label_font_dict)

ax.set_ylabel('$\Delta$ Dec [arcsec]', fontdict=helper.label_font_dict);

### Combined plot of all 3 images

In [ ]:
drange=1e-3

fig = plt.figure(figsize=(8, 2.3), dpi=300)
gs = fig.add_gridspec(ncols=5, nrows=1, width_ratios=[1, 1, 1, 0.1, 0.1], hspace=0.05)
axs = [fig.add_subplot(_gs) for _gs in gs]

for ax in axs[:-2]:
    ax.set_aspect('equal')
    ax.set_xlim(clip, -clip)
    ax.set_ylim(-clip, clip)
    ax.set_xlabel('$\Delta$ RA [arcsec]', fontdict=helper.label_font_dict)
    
# plot the two continuum images

for i, (ax, img, txt) in enumerate(zip(
        axs,
        [img_S, img_A, img_CO],
        ['scattered light (1.65 µm)', 'thermal emission (1.25 mm)', 'CO 2$-$1 peak intensity'])):
    
    if i < 2:
        # dust image
        cc1 = ax.pcolormesh(img.xi, img.yi, np.maximum(img.data.T, drange), norm=LogNorm(drange, 1.0), rasterized=True)
    else:
        # plot the gas image
        cc2 = ax.pcolormesh(img.xi, img.yi, np.maximum(img.data.T.max(0) / img.data.T.max(), 0.0), rasterized=True, cmap=mako)
    
    # scale bar
    ax.plot(-0.8 * clip * np.ones(2) + [0, bar_in_au / dpc], -0.82 * clip * np.ones(2), 'w', lw=3)
    ax.text(-0.8 * clip + bar_in_au / dpc / 2, -0.8 * clip, f'{bar_in_au:.0f} au',
            c='w', horizontalalignment='center', verticalalignment='bottom');
    ax.text(0.9 * clip, 0.9 * clip, txt, horizontalalignment='left', verticalalignment='top', c='w', fontsize='small')
    
    
# plot the rings in SPHERE

for i_ring in range(4):
    helper.plot_ring(axs[0], 
        r_HA[i_ring],
        hr_HA[i_ring] * r_HA[i_ring],
        np.deg2rad(inc_HA[i_ring]),
        np.deg2rad(90+PA_HA),
        nphi=50, c='w', alpha=0.25, lw=1, ls='--',  dashes=(5, 2))

# plot rings in ALMA
    
for i_ring in range(len(r_JH)):
    helper.plot_ring(axs[1], 
        r_JH[i_ring],
        0.0,
        np.deg2rad(inc_JH),
        np.deg2rad(90+PA_JH[i_ring]),
        nphi=50, c='w', alpha=0.25, lw=1, ls='--', dashes=(5, 2))


# adjust spacing
for ax in axs[1:3]:
    ax.yaxis.set_visible(False)
    
fig.subplots_adjust(wspace=0.02)
    
# colorbars
cb1 = plt.colorbar(cc1, cax=axs[-2])
cb2 = plt.colorbar(cc2, cax=axs[-1])
pos = axs[-3].get_position()
axs[-2].set_position([pos.x1 + 0.02, pos.y0, pos.width / 15., pos.height])
axs[-1].set_position([pos.x1 + 6 * pos.width / 15., pos.y0, pos.width / 15., pos.height])
cb2.set_label(r'$I_\nu / I_\nu^{max}$', fontdict=helper.label_font_dict)

# add ARAA-style labels

for t, ax in zip('abc', axs[:3]):
    pos = ax.get_position()
    fig.text(pos.x0, pos.y1 + 0.03, t, fontdict=helper.label_font_dict, c='k', size='large')

axs[0].set_ylabel('$\Delta$ Dec [arcsec]', fontdict=helper.label_font_dict);

In [ ]:
fig.savefig(helper.output_dir / 'imlup2.pdf', transparent=True, bbox_inches='tight')

## De-rotate IM Lup

We re-read the images with a larger FOV to be able to clip the rotated image

In [ ]:
clip = 4.5
PA = 90 + np.mean(PA_JH)  # derive a mean PA

img_S = helper.image(fname_S, clip=clip)
img_A = helper.image(fname_A, clip=clip)

# compute the slope of the intersecting line

a = 1/np.arctan(np.deg2rad(PA))

### Diagonally split image

In [ ]:
fig, ax = plt.subplots(dpi=200)
ax.set_aspect('equal')
ax.set_xlim(clip, -clip)
ax.set_ylim(-clip, clip)
ax.set_xlabel('$\Delta$ RA [arcsec]', fontdict=helper.label_font_dict)
    
# plot the two images

for i, (img, txt) in enumerate(zip([img_S, img_A], ['scattered light (1.65 $\mu$m)', 'thermal emission (1.25 mm)'])):

    # intersecting line
    _y = img.x * a
    plt.plot(img.x, _y)

    # mask above and below the line

    if i==0:
        mask = img.y[None, :] > _y[:, None]
    else:
        mask = img.y[None, :] < _y[:, None]
        
    #
    data = np.maximum(img.data, drange)
    data = np.where(mask, data, np.nan)

    
    # image
    cc = ax.pcolormesh(img.xi, img.yi, data.T, norm=LogNorm(drange, 1.0), rasterized=True)
    
    ax.text(0.9 * clip, (1 - 2 * i) * 0.9 * clip, txt, horizontalalignment='left', verticalalignment='top', c='w', fontsize='small')
    
# scale bar
ax.plot(-0.8 * clip * np.ones(2) + [0, bar_in_au / dpc], -0.82 * clip * np.ones(2), 'w', lw=3)
ax.text(-0.8 * clip + bar_in_au / dpc / 2, -0.8 * clip, f'{bar_in_au:.0f} au',
        c='w', horizontalalignment='center', verticalalignment='bottom');

    

# adjust spacing
axs[1].yaxis.set_ticklabels([])
fig.subplots_adjust(wspace=0.02)
    
# colorbar

pos = ax.get_position()
cax = fig.add_axes([pos.x1+0.02, pos.y0, pos.width / 15., pos.height])
cb = plt.colorbar(cc, cax=cax)
cb.set_label(r'$I_\nu / I_\nu^{max}$', fontdict=helper.label_font_dict)

ax.set_ylabel('$\Delta$ Dec [arcsec]', fontdict=helper.label_font_dict);

### De-rotated, split image

In [ ]:
from astropy.visualization import ImageNormalize

from astropy.visualization import PowerStretch

In [ ]:
drange = 0.005

In [ ]:
clip2 = 3.5

fig, ax = plt.subplots(dpi=200)
ax.set_aspect('equal')
ax.set_xlim(clip2, -clip2)
ax.set_ylim(-clip2, clip2)
ax.set_xlabel('$\Delta$ RA [arcsec]', fontdict=helper.label_font_dict)
    
# plot the two images

for i, (img, txt) in enumerate(zip([img_S, img_A], ['scattered light (1.65 µm)', 'thermal emission (1.25 mm)'])):

    _y = img.x * a

    if i==0:
        mask = img.y[None, :] > _y[:, None]
    else:
        mask = img.y[None, :] < _y[:, None]
        
    #
    data = np.maximum(img.data, drange)
    data = np.where(mask, data, np.nan)
    
    norm = ImageNormalize(data=data, stretch=PowerStretch(0.3), vmin=drange, vmax=1.0)
    
    # rotate
    _x, _y = np.meshgrid(img.xi, img.yi, indexing='ij')
    _xp = _x * np.cos(-np.deg2rad(PA-90)) - _y * np.sin(-np.deg2rad(PA-90))
    _yp = _x * np.sin(-np.deg2rad(PA-90)) + _y * np.cos(-np.deg2rad(PA-90))
    
    # image
    cc = ax.pcolormesh(_xp, _yp, data.T, norm=norm, rasterized=True)
    
    ax.text(0.9 * clip2 * (1 - 2 * i), 0.9 * clip2, txt,
            horizontalalignment='left' * (i==0) + 'right' * (i==1), verticalalignment='top',
            c='w', fontsize='x-small')
    
# scale bar
ax.plot(-0.8 * clip2 * np.ones(2) + [0, bar_in_au / dpc], -0.82 * clip2 * np.ones(2), 'w', lw=3)
ax.text(-0.8 * clip2 + bar_in_au / dpc / 2, -0.8 * clip2, f'{bar_in_au:.0f} au',
        c='w', horizontalalignment='center', verticalalignment='bottom');
    

# adjust spacing
axs[1].yaxis.set_ticklabels([])
fig.subplots_adjust(wspace=0.02)
    
# colorbar

pos = ax.get_position()
cax = fig.add_axes([pos.x1+0.02, pos.y0, pos.width / 15., pos.height])
cb = plt.colorbar(cc, cax=cax, ticks=[0.005, 0.01, 0.1, 0.2, 0.4, 1.0])
cb.set_label(r'$I_\nu / I_\nu^{max}$', fontdict=helper.label_font_dict)

ax.set_ylabel('$\Delta$ Dec [arcsec]', fontdict=helper.label_font_dict);
ax.set_facecolor('k')

In [ ]:
fig.savefig(helper.output_dir / 'IMLup-2face.pdf', bbox_inches='tight')